<a href="https://colab.research.google.com/github/jopapo/nuinvest_options_irpf/blob/main/A%C3%A7%C3%B5es_NuInvest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Not installed libs

!pip install pdfminer

In [31]:
# Requirements

import requests
from pathlib import Path
from getpass import getpass

In [26]:
# Variables
year = 2021

access_token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJuYmYiOjE2NDcwNTY4NjUsImV4cCI6MTY0NzA4NTY2NSwiaXNzIjoiaHR0cHM6Ly9hcGkuZWFzeW52ZXN0LmNvbS5ici9hdXRoIiwiYXVkIjpbImh0dHBzOi8vYXBpLmVhc3ludmVzdC5jb20uYnIvYXV0aC9yZXNvdXJjZXMiLCJodHRwczovL3d3dy5lYXN5bnZlc3QuY29tLmJyIl0sImNsaWVudF9pZCI6Ijg3NmRhYjIxOTA0NjQ4ODRiZjliMDkyYWExNDA3NTg1Iiwic3ViIjoiMDA3NzE1MzA5MDAiLCJhdXRoX3RpbWUiOjE2NDcwNTY4NjUsImlkcCI6Imh0dHBzOi8vYXBpLmVhc3ludmVzdC5jb20uYnIvYXV0aCIsInVuaXF1ZV9uYW1lIjoiMDA3NzE1MzA5MDAiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9uYW1laWRlbnRpZmllciI6IjAwNzcxNTMwOTAwIiwibmFtZWlkIjoiMDA3NzE1MzA5MDAiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9naXZlbm5hbWUiOiJKb2FvIFBhdWxvIFBvZmZvIiwiZ2l2ZW5fbmFtZSI6IkpvYW8gUGF1bG8gUG9mZm8iLCJ2ZXJzaW9uIjoidjEiLCJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9lbWFpbGFkZHJlc3MiOiJqb3BhcG9AZ21haWwuY29tIiwiZW1haWwiOiJqb3BhcG9AZ21haWwuY29tIiwiYWNjIjoiNjgzMjI1MSIsImFkdiI6IjgwIiwiZXN0ayI6Imluc3RhbGxlZCIsImNsaSI6IntcIkNsaWVudElkXCI6XCI4NzZkYWIyMTkwNDY0ODg0YmY5YjA5MmFhMTQwNzU4NVwiLFwiTmFtZVwiOlwiUG9ydGFsL0hvbWUgQnJva2VyXCIsXCJJbnRlcm5hbFwiOmZhbHNlfSIsImNpYSI6Ijo6ZmZmZjoxMC42NS43NS4xMjYiLCJkZXZpY2VfaWQiOiJhYTVjNmFjMmNlNDYxMmVkMDVkMTllYTc4MmVmZWExZiIsInNjb3BlIjpbIm9wZW5pZCIsImh0dHBzOi8vd3d3LmVhc3ludmVzdC5jb20uYnIiLCJvZmZsaW5lX2FjY2VzcyJdLCJhbXIiOlsicGFzc3dvcmQiXX0.GrjyZcfiwg7Tc68wB_cIusIGTxysSXpkQ_A2pFUX0OA"

session = requests.Session()
session.headers.update({'Authorization': f"Bearer {access_token}"})


In [27]:
# Get Invoices Pdfs

def get_invoices(prefix):

  response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/1/{prefix}?startDate={year}-01-01&endDate={year}-12-31")
  response.raise_for_status()

  print(prefix, response)

  invoices = []
  if response.status_code == 204:
    print(f'No data for {prefix}')
  else:
    invoices = response.json()['value']['invoices']
  
  for invoice in invoices:
    params_values = {key:val for (key,val) in invoice.items() if key in ['invoiceNumber', 'custodyId', 'date']}
    # FI não tem custodyId
    # RV tem o data e não tem custodyid
    # Não testado com TP e PS pq eu não tinha papéis pra isso

    response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/report/1/{prefix}", params=params_values, stream=True)
    response.raise_for_status()

    filename = Path(f"Invoice_{prefix}_" + '_'.join(str(x) for x in params_values.values()) + '.pdf')
    filename.write_bytes(response.content)

papers = ['TD', 'TP', 'FI', 'RV', 'PS']
for paper in papers:
  get_invoices(paper)



TD <Response [200]>
TP <Response [204]>
No data for TP
FI <Response [200]>
RV <Response [200]>
PS <Response [204]>
No data for PS


In [15]:
# Process all pdfs to txt
%%bash
for f in *.pdf
do
 pdf2txt.py -o $f.txt $f
done